In [1]:
### Import important libraries/modules

In [2]:
#!/usr/bin/env python2.7
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
from scipy.interpolate import Rbf, griddata, interp2d
import json
import csv
import numpy as np

In [3]:
import sys
import requests
import datetime

In [4]:
## read the csv to fetch list of stations with respective lat-longs in our csv file
def get_station_xyz(filename):
    x = []; y = []; z = []
    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter='|')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                print('Column names are %s' % row)
                line_count += 1
            else:
                x.append(float(row[2]))
                y.append(float(row[1]))
               # z.append(float(row[4]))
                line_count += 1

    return x,y

(x,y)=get_station_xyz('station_locations.csv')
print([x,y])

    

Column names are ['name', 'latitude', 'longitude', 'code', 'temp', '', '', '', '', '']
[[144.98, 144.8, 144.48, 145.18, 145.41, 144.91, 144.93, 145.34, 145.12, 144.38, 144.76, 145.1, 145.31, 145.26, 144.13, 144.8, 144.96, 145.1], [-37.83, -37.67, -38.03, -38.36, -37.72, -37.73, -37.95, -37.87, -38.15, -38.17, -37.86, -37.98, -38.46, -37.87, -37.91, -38.31, -37.86, -37.84]]


In [5]:
def contexts():
    headers={'User-Agent':'Mozilla/5.0'}
    url_front="http://www.bom.gov.au/fwo/IDV60901/IDV60901."
    url_end=".json"
    urls_middle=["95936","94866","94854","94898","94864","95866","95872","94872","94871","94857","94865","94870","94892","95867","94863","94853","94864","95874"]
    #places=["Melbourne (Olympic Park)","Melbourne Airport","Avalon Airport", "Cerberus", "Coldstream", "Essendon Airport", "Fawkner Beacon","Ferny    Creek","Frankston","Geelong Racecourse","Laverton","Moorabbin Airport","Rhyll","Scoresby","Sheoaks","South Channel Island","St Kilda Harbour","Viewbank"]

    id=0;
    temp_array=[]
    curr_array=[]
    time_array=[]
    df=[]
    d1=[]
    for mid in urls_middle:
        id=id+1
        url_full=url_front+mid+url_end
    #print(url_full)
        htmltext = requests.get(url_full)
        data=htmltext.json()
        temp_array=[]
        for i in data["observations"]["data"]:
            temp_array.append(i["apparent_t"])
        #temp_array.append(data["observations"]["data"][0]["apparent_t"])
        #curr_t=data["observations"]["data"][0]["air_temp"]
            
            new=datetime.datetime.strptime(i["aifstime_utc"],"%Y%m%d%H%M%S")
            formatted=new.strftime('%Y-%m-%d %H:%M:%S')
            time_array.append(formatted)

    #        dd2=response.query.results.observations.header;
    #           for(i=0;i<dd.length;i++)
    #        {times.push(dd[i].local_date_time_full);
    #        appT.push(dd[i].apparent_t);
    #        currT.push(dd[i].air_temp)
        a = map(lambda x: '' if x == None else x, temp_array)
        d1= [mid,temp_array,time_array]
        
        df.append(d1)
    return df, data["observations"]["data"]

df,d=contexts()
temp=[]
time=[]
for i in df:
    temp.append(i[1][0])
    time.append(i[1][0])
t=[0 if v is None else v for v in temp]  # replace nans with 0
ave=sum(t)/len(t)
t=[ave if v is 0 else v for v in t] # now replace 0s with average -- needed for better interpolation.


In [6]:
## Calculate longitude and latitude max/mins
LONG_LIMITS=[float(min(y)), float(max(y))]
LAT_LIMITS=[float(min(x)),float(max(x))]


In [7]:
def get_meshgrid(x,y,z,LONG_LIMITS,LAT_LIMITS,gridsize):
    gridsize=20

    XI, YI = np.meshgrid(
        np.linspace(LAT_LIMITS[0], LAT_LIMITS[1], gridsize),
        np.linspace(LONG_LIMITS[0], LONG_LIMITS[1], gridsize)
        )
    rbf = Rbf(x, y, z, method='nearest')  # uses SCIPY Interpolate RBF - Radial Basis Function - nearest neighbour mode for interpolation
    ZI = rbf(XI, YI)
    return (x, y, z), (XI, YI, ZI)

(x,y,t),(XI,YI,ZI)=get_meshgrid(x,y,t,LONG_LIMITS,LAT_LIMITS,gridsize=20)

#print(ZI)
# determine way to print data in the format we use for the d3js graph
newline=ZI.flatten()
count=0;
t=[]
for i in newline:
    t.append(count)
    count+=1
    
print(t)
    
print(list(newline))
with open('file_to_write', 'w') as f:
    f.write('%s' % t)
    f.write('%s' % list(newline))
    
f.close()


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [9]:
def plot_rbf(x, y, z, XI, YI, ZI):
    n = plt.Normalize(-2., 2.)
    plt.subplot(1, 1, 1)
    plt.pcolor(XI, YI, ZI, cmap=cm.jet)
    plt.scatter(x, y, 100, z, cmap=cm.jet)
    plt.title('RBF interpolation - multiquadrics')
    #plt.xlim(-2, 2)
    #plt.ylim(-2, 2)
    plt.colorbar()
    plt.show()
    
plot_rbf(x, y, t, XI, YI, ZI)

ValueError: to_rgba: Invalid rgba arg "0"
to_rgb: Invalid rgb arg "0"
cannot convert argument to rgb sequence

In [ ]:
def plot_griddata(grid_z, points,x_lim=LONG_LIMITS, y_lim=LAT_LIMITS, title='Linear'):
    plt.subplot(121)
    #plt.imshow(func(grid_x, grid_y).T, extent=(0,1,0,1), origin='lower')
    n = plt.Normalize(-2., 2.)
    plt.plot(points[:,0], points[:,1], 'k.', ms=1)
    plt.title('Original')
    plt.subplot(122)
    plt.imshow(grid_z.T, extent=(x_lim[0],x_lim[1], y_lim[0], y_lim[1]), origin='lower')
    plt.title(title)
    # plt.subplot(223)
    # plt.imshow(grid_z1.T, extent=(x_lim[0],x_lim[1], y_lim[0], y_lim[1]), origin='lower')
    # plt.title('Linear')
    # plt.subplot(224)
    # plt.imshow(grid_z2.T, extent=(x_lim[0],x_lim[1], y_lim[0], y_lim[1]), origin='lower')
    # plt.title('Cubic')
    plt.gcf().set_size_inches(6, 6)

    plt.show()
    


In [ ]:
#def get_griddata(points_per_degree=3,LONG_LIMITS,LAT_LIMITS):
#    x, y, z = get_station_xyz('station_locations.csv')         
#    points = np.array([x, y]).T
#    grid_x, grid_y = np.mgrid[LONG_LIMITS[0]:LONG_LIMITS[1]:complex(points_per_degree * (LONG_LIMITS[1] - LONG_LIMITS[0])), LAT_LIMITS[0]:LAT_LIMITS[1]:complex(points_per_degree * (LAT_LIMITS[1] - LAT_LIMITS[0]))]
#    grid_z = griddata(points, np.array(z), (grid_x, grid_y), method='linear')
#    print('Grid shape: %s from %d station readings'%(repr(grid_z.shape), len(x))) 
#    return grid_x, grid_y, grid_z, points

#grid_x, grid_y, grid_z, points = get_griddata()
#plot_griddata(z, points)


In [ ]:
#im!{sys.executable} -m pip install geojson